# Lab Three: Association Rule Mining, Clustering, or Collaborative Filtering
### Laura Ahumada, Kevin Boyd, Shikha Pandey, Duy Nguyen
##### Southern Methodist University
##### Fall, 2022

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

#Reading the files
df_movies = pd.read_csv('movies.csv')
df_ratings = pd.read_csv('ratings.csv')
df_links = pd.read_csv('links.csv')
df_tags = pd.read_csv('tags.csv')

#Merging the Dataset
df_merge1=df_movies.merge(df_ratings,how="outer",on="movieId")
df_final=df_merge1.merge(df_links,how="outer",on="movieId")
df_final.groupby("movieId").mean("rating")

In [45]:
all_=[]
for i in df_final["genres"]:
    l=(i.split("|"))
    for j in l:
        all_.append(j)
set(all_)

{'(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

In [ ]:
df_final.to_csv("movies_final.csv")

In [2]:
movies_final = pd.read_csv('movies_final.csv')